# Replication of Hemker (2018)

The goal of this notebook is to follow the methodology explained in Hemker (2018) to perform a replication of his results. Note that the source code is not available, rendering this task a bit harder.

### Data Retrieval

In [25]:
# Source: Davidson et al. (2017)

import pandas as pd

df = pd.read_csv("./data/labeled_data.csv", index_col=0)
tweets = df.tweet
labels = df["class"].values

In [58]:
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


## Data Preprocessing
---

In [64]:
# Source: Davidson et al. (2017)

import re
import html

def preprocess(text_string):
    
    # Regex
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    
    # If we wish to find hashtags, we have to unescape HTML entities
    parsed_text = html.unescape(text_string)
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    
    return parsed_text

def _test_preprocess():
    
    assert "HASHTAGHERE" == preprocess("#iam1hashtag")
    assert "URLHERE" == preprocess("https://seminar.minerva.kgi.edu")
    assert "MENTIONHERE" == preprocess("@vinimiranda")
    assert ' ' == preprocess("        ")
    assert "&MENTIONHERE URLHERE HASHTAGHERE " == \
        preprocess("&amp;@vinimiranda    https://seminar.minerva.kgi.edu     #minerva    ")
    
_test_preprocess()



In [57]:
preprocess(tweets[68])

'"MENTIONHERE: MENTIONHERE MENTIONHERE bitch fuck u URLHERE" maybe you\'ll get better. Just URLHERE'